In [ ]:
from google.colab import files
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from time import time
start_nb = time()

start = time()
import pandas as pd
from nltk.corpus import stopwords
import re
from nltk import word_tokenize

es_stops = set(stopwords.words('spanish'))

print('Importación de Librerias. %.2f segundos' % (time() - start))

Importación de Librerias. 0.00 segundos


In [ ]:
start = time()
dataset = pd.read_csv('//content/gdrive/My Drive/df.csv',
                      sep=';', encoding='utf-8-sig')
dataset_size = len(dataset)


print('Dataset generado. %.2f segundos' % (time() - start))
dataset.head()

Dataset generado. 1.47 segundos


,Id,Title,Body
0,286498,Debugear código Django utilizando Docker,<p>Les cuento.\r\nEstoy realizando un proyecto...
1,226916,En el file.conf muestra Error usando Superviso...,<p>Al configurar el file.conf<br>\r\nruta : su...
2,161561,Ejecutar navegador en pantalla completa al ini...,<p>Estoy realizando un <em>proyecto Domotico</...
3,230560,Pagina web en blanco,<p>Buen dia \r\nNo se por que mi pagina web sa...
4,309146,Obtengo el error I/Choreographer: Skipped 110 ...,<p>Estoy trabajando en android studio intentan...


In [ ]:
def preprocess(doc):
    label = re.compile('<.*?>')
    doc = re.sub(label, '', doc)
    doc = doc.lower()  # Minuscula todo el texto.
    doc = word_tokenize(doc)  # Dividir en palabras.
    doc = [w for w in doc if not w in es_stops]  # Eliminar stopwords.
    doc = [w for w in doc if w.isalpha()]  # Eliminar numbers and punctuation.
    return doc

In [ ]:
start = time()
dataset['Processed_Title'] = dataset.apply(
        lambda dataset: preprocess(dataset['Title']), axis=1)
dataset['Processed_Body'] = dataset.apply(
        lambda dataset: preprocess(dataset['Body']), axis=1)
print('Preprocesado completo. %.2f segundos' % (time() - start))
dataset.iloc[:,[3,4]]

Preprocesado completo. 1.57 segundos


,Processed_Title,Processed_Body
0,"[debugear, código, django, utilizando, docker]","[cuento, realizando, proyecto, django, utilizo..."
1,"[muestra, error, usando, supervisor, digital, ...","[configurar, ruta, sudo, nano, program, factur..."
2,"[ejecutar, navegador, pantalla, completa, inic...","[realizando, proyecto, domotico, siguientes, c..."
3,"[pagina, web, blanco]","[buen, dia, pagina, web, sale, blanco, alguien..."
4,"[obtengo, error, skipped, frames, the, applica...","[trabajando, android, studio, intentando, hace..."
...,...,...
1079,"[conexion, remota, sql, server, linux, mint]","[instalado, sql, server, linux, mint, base, da..."
1080,"[crear, base, datos, centos, funciona]","[utilizando, centos, seguí, instrucciones, sit..."
1081,"[hacer, respaldo, base, datos, terminal]","[base, datos, ms, sql, server, express, ubuntu..."
1082,"[aplicación, windows, linux, wine]","[aplicación, desarrollada, utiliza, cámara, es..."


In [ ]:
from sklearn.model_selection import train_test_split 

In [ ]:
train_p, test_p = train_test_split(dataset['Processed_Title'], test_size = 0.20, shuffle = False)

print("Ejemplos usados para entrenar: ", len(train_p))
print("Ejemplos usados para test: ", len(test_p))

Ejemplos usados para entrenar:  867
Ejemplos usados para test:  217


In [ ]:
processed_title = dataset['Processed_Title'].to_list()

title = dataset['Title'].to_list()

In [ ]:
from gensim.models import Word2Vec
# Train Word2Vec on all the restaurants.
model_p = Word2Vec(processed_title, workers=3, size=100)

# Inicialización de WmdSimilarity.
from gensim.similarities import WmdSimilarity
num_best = 5
instancia_p = WmdSimilarity(train_p, model_p, num_best=5)
print(instancia_p)

WmdSimilarity<867 docs, 100 features>


In [ ]:
start = time()

msj_p = 'Error “bad magic number” al cifrar/descifrar utilizando OpenSSL'
bus_p = preprocess(msj_p)
print(bus_p)
sim_p = instancia_p[bus_p]
# print(len(sim_p)) 5
print ('Cell took %.2f seconds to run.' %(time() - start))

['error', 'bad', 'magic', 'number', 'utilizando', 'openssl']
Cell took 0.18 seconds to run.


In [ ]:
print ('Busqueda: ', msj_p)
print("")
for i in range(num_best): # (num_best)
    print (title[sim_p[i][0]],'---> sim = %.4f' % sim_p[i][1])

Busqueda:  Error “bad magic number” al cifrar/descifrar utilizando OpenSSL

Error "bad magic number" al cifrar/descifrar utilizando OpenSSL ---> sim = 1.0000
Error al implementar generación aleatoria de números en C ---> sim = 0.6065
¿Por qué me tira este error de enlazado? ---> sim = 0.6063
Error: std::out_of_range ---> sim = 0.6063
Error MPI en C++ ---> sim = 0.6063
